In [ ]:
# %pip install split-folders

In [ ]:
import numpy as np
import tensorflow as tf
import os
from splitfolders import ratio
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
base_plant = "..\\tmp\plant-cv-images"

ratio('D:\Dataset\Tanaman\Versi 4\\Botanify\\dataset', output='..\\tmp\plant-cv-images', seed=1337, ratio=(.6, .4))
train_dir_plant = os.path.join(base_plant, 'train')
val_dir_plant = os.path.join(base_plant, 'val')

In [ ]:
os.listdir( train_dir_plant)

In [ ]:
os.listdir(val_dir_plant)

In [ ]:
train_plant_datagen = ImageDataGenerator(
  rescale=1./255,
  rotation_range=20,
  horizontal_flip=True,
  shear_range=0.2,
  fill_mode='nearest'
)

test_plant_datagen = ImageDataGenerator(
  rescale=1./255
)

In [ ]:
train_plant_generator = train_plant_datagen.flow_from_directory(
  train_dir_plant,
  target_size=(224, 224),
  batch_size=32,
  class_mode='categorical'
)

validation_plant_generator = test_plant_datagen.flow_from_directory(
  val_dir_plant,
  target_size=(224, 224),
  batch_size=32,
  class_mode='categorical'
)

In [ ]:
# 1st model
# model = Sequential([
#     ZeroPadding2D(padding=(2,2), input_shape=(150, 150, 3)),
#     Conv2D(16, (5,5), strides=(3, 3), activation='relu'),
#     MaxPooling2D(2, 2),
#     Conv2D(32, (3, 3), strides=(1, 1), activation='relu'),
#     ZeroPadding2D(padding=(1,1)),
#     Conv2D(64, (3, 3), strides=(1, 1), activation='relu'),
#     Flatten(),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(29, activation='softmax')
# ])

In [ ]:
# 2nd model
model = Sequential([
    Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((3, 3), strides=(2, 2)),
    Conv2D(256, (5, 5), padding='same', activation='relu'),
    MaxPooling2D((3, 3), strides=(2, 2)),
    Conv2D(384, (3, 3), padding='same', activation='relu'),
    Conv2D(384, (3, 3), padding='same', activation='relu'),
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((3, 3), strides=(2, 2)),
    Flatten(),
    Dense(4096, activation='relu'),
    Dense(4096, activation='relu'),
    Dense(29, activation='softmax')
])

In [ ]:
# compile model dengan 'adam' optimizer loss function 'categorical_crossentropy' 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.0001),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 1.0) and (logs.get('val_a ccuracy') > 9.0):
      print("\nAkurasi telah mencapai > 100%!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
history = model.fit(
  train_plant_generator,
  epochs=9,
  validation_data=validation_plant_generator,
  verbose=1,
  callbacks=[callbacks]
)

In [ ]:
plt.plot(history.history['accuracy'], label='Training')
plt.plot(history.history['val_accuracy'], label='Testing')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Testing')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()

In [ ]:
model.evaluate(validation_plant_generator, verbose=2)

In [ ]:
class_name = os.listdir(val_dir_plant)
class_name.sort()

In [ ]:
test_folder = os.listdir('D:/Dataset/unseen/')

score_result = {} 

total_score = 0
total_data = 0

for folder in test_folder:
  score = 0
  i = 0
  plant = os.listdir(f'D:/Dataset/unseen/{folder}')

  for uploaded in plant:
    fn = f'D:/Dataset/unseen/{folder}/{uploaded}'
    # predicting images
    path = fn
    img = image.load_img(path, target_size=(224,224))

    # imgplot = plt.imshow(img)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])

    classes = model.predict(images, batch_size=10)
    out = np.argmax(classes)
    # print(classes)
    # print(class_name)
    print(class_name[out])
    if class_name[out] == folder:
      score += 1
    i += 1
    total_data += 1

  total_score += score
  score_result[folder] = f'{score}/{i}'

print(f"Score Result: {score_result}\n Total Score: {total_score}/{total_data} ({(total_score/total_data)*100}%)")